# Create Agents

In [2]:
from mesa import Agent
import random

class BorrowerAgent(Agent):
    def __init__(self, model, collateral, borrow_amount):
        super().__init__(model)
        self.collateral = collateral  # Amount of collateral provided
        self.borrow_amount = borrow_amount  # Amount the agent wants to borrow
        self.debt = 0  # Current outstanding debt

    def step(self):
        # Simulate borrowing behavior
        if self.debt == 0 or self.collateral > 0:
            self.borrow()

        # Simulate repaying behavior
        if self.debt > 0 and random.random() < 0.5:  # 50% chance of repaying some debt
            self.repay()

    def borrow(self):
        """Attempt to borrow based on risk tolerance and collateral."""
        max_borrow = self.collateral * self.model.loan_to_value_ratio
        if self.borrow_amount <= max_borrow and self.borrow_amount > 0:
            self.debt += self.borrow_amount
            self.model.total_borrowed += self.borrow_amount
            print(f"Borrower {self.unique_id} borrowed {self.borrow_amount}")

    def repay(self):
        """Repay a portion of the debt."""
        repayment = min(self.debt, random.uniform(0.1, 0.3) * self.debt)  # Repay 10-30% of debt
        self.debt -= repayment
        self.model.total_borrowed -= repayment
        print(f"Borrower {self.unique_id} repaid {repayment}, remaining debt: {self.debt}")


class LenderAgent(Agent):
    def __init__(self, model, liquidity):
        super().__init__(model)
        self.liquidity = liquidity  # Amount of funds available to lend

    def step(self):
        # Simulate lending behavior
        self.supply_liquidity()

    def supply_liquidity(self):
        """Supply liquidity to the AAVE protocol."""
        if self.liquidity > 0:
            self.model.total_liquidity += self.liquidity
            print(f"Lender {self.unique_id} supplied {self.liquidity}")
            self.liquidity = 0  # Assume full supply for simplicity


KeyboardInterrupt: 

In [ ]:
from mesa import Model
from mesa.datacollection import DataCollector
import random

class AAVEModel(Model):
    def __init__(self, num_borrowers, num_lenders, collateral, borrow_amount, initial_liquidity, loan_to_value_ratio, liquidation_threshold, liquidation_bonus, base_rate, rate_slope):
        super().__init__()
        self.steps = 0  # Internal step counter
        self.total_liquidity = initial_liquidity  # Total liquidity in the lending pool
        self.total_borrowed = 0  # Total borrowed funds
        self.loan_to_value_ratio = loan_to_value_ratio  # LTV ratio (e.g., 0.75)
        self.liquidation_threshold = liquidation_threshold  # Threshold for liquidation (e.g., 0.85)
        self.liquidation_bonus = liquidation_bonus  # Bonus for liquidators (e.g., 0.1)
        self.base_rate = base_rate  # Base interest rate (e.g., 0.02 for 2%)
        self.rate_slope = rate_slope  # Slope for utilization-based rate changes

        # Create agents
        BorrowerAgent.create_agents(model=self, collateral=collateral, borrow_amount=borrow_amount, n=num_borrowers)
        LenderAgent.create_agents(model=self, liquidity=initial_liquidity, n=num_lenders)

        # Data collection
        self.datacollector = DataCollector(
            model_reporters={
                "Total Liquidity": lambda m: m.total_liquidity,
                "Total Borrowed": lambda m: m.total_borrowed,
                "Utilization Rate": lambda m: m.total_borrowed / m.total_liquidity,
                "Interest Rate": lambda m: m.calculate_interest_rate()
            }
        )

    def step(self):
        """Advance the model by one step."""
        self.steps += 1
        self.datacollector.collect(self)
        # First activate all sheep, then all wolves, both in random order
        self.agents_by_type[BorrowerAgent].shuffle_do("step")
        self.agents_by_type[LenderAgent].shuffle_do("step")

        # Check for liquidations after agents act
        self.check_liquidations()

    def calculate_interest_rate(self):
        """Calculate dynamic interest rate based on utilization."""
        if self.total_liquidity == 0:
            return self.base_rate  # Prevent division by zero
        utilization = self.total_borrowed / self.total_liquidity
        return self.base_rate + (self.rate_slope * utilization)

    def check_liquidations(self):
        """Liquidate borrowers who exceed the liquidation threshold."""
        for borrower in self.agents_by_type[BorrowerAgent]:
            if borrower.debt > borrower.collateral * self.liquidation_threshold:
                self.liquidate(borrower)

    def liquidate(self, borrower):
        """Seize a portion of collateral and repay some of the debt."""
        seized_collateral = borrower.collateral * (1 + self.liquidation_bonus)
        repayment = min(seized_collateral, borrower.debt)
        
        borrower.debt -= repayment
        borrower.collateral -= seized_collateral  # Collateral is reduced
        self.total_liquidity += repayment  # Repaid amount is added back to liquidity
        self.total_borrowed -= repayment  # Reduce total borrowed amount

        print(f"Borrower {borrower.unique_id} was liquidated! Seized {seized_collateral} collateral, repaid {repayment} debt.")


In [ ]:
model = AAVEModel(num_borrowers=20, 
                  num_lenders=15, 
                  collateral=100, 
                  borrow_amount=50, 
                  initial_liquidity=1000, 
                  loan_to_value_ratio=0.75, 
                  liquidation_threshold=0.85, 
                  liquidation_bonus=0.1, 
                  base_rate=0.02, 
                  rate_slope=0.01)

In [ ]:
for i in range(30):
    model.step()

Borrower 2 borrowed 50
Borrower 2 repaid 5.570884431131475, remaining debt: 44.429115568868525
Borrower 9 borrowed 50
Borrower 9 repaid 6.272823650361993, remaining debt: 43.72717634963801
Borrower 17 borrowed 50
Borrower 17 repaid 11.820730080918846, remaining debt: 38.17926991908115
Borrower 5 borrowed 50
Borrower 8 borrowed 50
Borrower 18 borrowed 50
Borrower 18 repaid 10.58639169064644, remaining debt: 39.41360830935356
Borrower 1 borrowed 50
Borrower 1 repaid 6.5472288190757215, remaining debt: 43.45277118092428
Borrower 19 borrowed 50
Borrower 19 repaid 12.144373132930381, remaining debt: 37.85562686706962
Borrower 3 borrowed 50
Borrower 6 borrowed 50
Borrower 6 repaid 5.078434011560195, remaining debt: 44.9215659884398
Borrower 20 borrowed 50
Borrower 11 borrowed 50
Borrower 10 borrowed 50
Borrower 16 borrowed 50
Borrower 16 repaid 7.961377941543956, remaining debt: 42.03862205845604
Borrower 4 borrowed 50
Borrower 14 borrowed 50
Borrower 12 borrowed 50
Borrower 12 repaid 9.9864

In [ ]:
model.datacollector.get_model_vars_dataframe()

,Total Liquidity,Total Borrowed,Utilization Rate,Interest Rate
0,1000.000000,0.000000,0.000000,0.020000
1,16000.000000,895.319111,0.055957,0.020560
2,16745.248776,882.418807,0.052697,0.020527
3,17922.462796,182.096792,0.010160,0.020102
4,17799.262796,276.714738,0.015546,0.020155
5,17811.582796,233.302281,0.013098,0.020131
6,17810.350796,224.794576,0.012622,0.020126
7,17810.473996,188.103910,0.010561,0.020106
8,17810.461676,180.335586,0.010125,0.020101
9,17810.462908,159.273135,0.008943,0.020089
